# Fitness Trackers

### 1. Uno de los primeros pasos para realizar un buen análisis de datos es familiarizarnos con los datos que contiene el fichero a analizar. Para ello, calcularemos los estadísticos descriptivos elementales de las variables del fichero. Una vez cargados los datos en nuestro programa Python (utilizando la librería Pandas), calcula los siguientes valores para cada una de las variables:

In [375]:
import math
import pandas as pd

datos = pd.read_csv("Fitness_trackers.csv")
datos.head()

,Brand Name,Device Type,Model Name,Color,Selling Price,Original Price,Display,Rating (Out of 5),Strap Material,Average Battery Life (in days),Reviews
0,Xiaomi,FitnessBand,Smart Band 5,Black,"2,499","2,999",AMOLED Display,4.1,Thermoplastic polyurethane,14,NaN
1,Xiaomi,FitnessBand,Smart Band 4,Black,"2,099","2,499",AMOLED Display,4.2,Thermoplastic polyurethane,14,NaN
2,Xiaomi,FitnessBand,HMSH01GE,Black,"1,722","2,099",LCD Display,3.5,Leather,14,NaN
3,Xiaomi,FitnessBand,Smart Band 5,Black,"2,469","2,999",AMOLED Display,4.1,Thermoplastic polyurethane,14,NaN
4,Xiaomi,FitnessBand,Band 3,Black,"1,799","2,199",OLED Display,4.3,Plastic,7,NaN


* Número de muestras (valores distintos de missing) 

In [376]:
nMuestras = []
for column in datos.columns:
    nMuestras.append([column, len(datos[column].dropna())])
nMuestras

[['Brand Name', 565],
 ['Device Type', 565],
 ['Model Name', 565],
 ['Color', 565],
 ['Selling Price', 565],
 ['Original Price', 565],
 ['Display', 565],
 ['Rating (Out of 5)', 514],
 ['Strap Material', 565],
 ['Average Battery Life (in days)', 565],
 ['Reviews', 78]]

Se comprueba que 'Rating (Out of 5)' y 'Reviews' tienen valores _missing_.

* Media y desviación estándar de aquellas variables en las que tenga sentido (numéricas)
* Valor mínimo y valor máximo de aquellas variables en las que tenga sentido (numéricas)

In [377]:
datos.dtypes

Brand Name                         object
Device Type                        object
Model Name                         object
Color                              object
Selling Price                      object
Original Price                     object
Display                            object
Rating (Out of 5)                 float64
Strap Material                     object
Average Battery Life (in days)      int64
Reviews                            object
dtype: object

Si comparamos los tipos asignados con los numéricos, vemos que además del Rating y la Average Battery Life también deberían ser numéricos los precios. Si buscamos precios de los trackers en internet, nos daremos cuenta de que son erróneos:
* Si están en formato americano (coma = miles), el precio está multiplicado por 10000.
* Si están en formato europeo (coma = decimales), el precio está dividido por 10.

La intención es cambiar estos tipos a numérico y corregir precios, sin embargo sigue habiendo valores anómalos (dos comas). Primero corregiremos estos.

In [378]:
preciosVenta = []
preciosOrig = []


for element in datos["Selling Price"]:
    if element.count(",")!=1: #Primero corregimos valores con dos comas quitando la primera
        posPrimeraComa = element.find(",")
        precio = element[:posPrimeraComa] + element[posPrimeraComa+1:]
        
#     posComa = element.find(",")
#     #Version1 = corregir precio
# #     precioConPunto = element[:posComa+2] + "." + element[posComa+2:]
# #     precio = precioConPunto.replace(",","")
#     #Version2 (current)= corregir coma
    precio = element.replace(",","")
    preciosVenta.append(precio)
     
datos["Selling Price"] = pd.to_numeric(preciosVenta, downcast="integer")

for element in datos["Original Price"]:
    if element.count(",")!=1: #Primero corregimos valores con dos comas quitando la primera
        posPrimeraComa = element.find(",")
        precio = element[:posPrimeraComa] + element[posPrimeraComa+1:]
        
    precio = element.replace(",","")        
    preciosOrig.append(precio)

datos["Original Price"] = pd.to_numeric(preciosOrig, downcast="integer")

    
datos.head(10)


,Brand Name,Device Type,Model Name,Color,Selling Price,Original Price,Display,Rating (Out of 5),Strap Material,Average Battery Life (in days),Reviews
0,Xiaomi,FitnessBand,Smart Band 5,Black,2499,2999,AMOLED Display,4.1,Thermoplastic polyurethane,14,NaN
1,Xiaomi,FitnessBand,Smart Band 4,Black,2099,2499,AMOLED Display,4.2,Thermoplastic polyurethane,14,NaN
2,Xiaomi,FitnessBand,HMSH01GE,Black,1722,2099,LCD Display,3.5,Leather,14,NaN
3,Xiaomi,FitnessBand,Smart Band 5,Black,2469,2999,AMOLED Display,4.1,Thermoplastic polyurethane,14,NaN
4,Xiaomi,FitnessBand,Band 3,Black,1799,2199,OLED Display,4.3,Plastic,7,NaN
5,Xiaomi,FitnessBand,Band - HRX Edition,Black,1299,1799,OLED Display,4.2,Plastic,20,NaN
6,Xiaomi,FitnessBand,Band 2,Black,2499,2499,OLED Display,4.3,Plastic,7,NaN
7,Xiaomi,Smartwatch,Revolve,Black,12349,15999,AMOLED Display,4.4,Silicone,14,2
8,Xiaomi,Smartwatch,RevolveActive,Black,12999,15999,AMOLED Display,4.4,Silicone,14,3
9,Xiaomi,FitnessBand,Smart Band 3i,Black,1270,1599,OLED Display,4.2,Thermoplastic polyurethane,7,NaN


Además, el número de reviews tampoco está en tipo numérico. Si bien no es de utilidad calcular estadísticos generales sobre esta variable, la convertiremos en numérica para posteriores cálculos concretos:

In [379]:
reviewsOrig = []

for element in datos["Reviews"]:
    if type(element) is str:
        posComa = element.find(",")
        review = element[:posComa] + element[posComa+1:]
        review = element.replace(",","")
        element = review

    reviewsOrig.append(element)

datos["Reviews"] = pd.to_numeric(reviewsOrig) # TODO pasar a entero

datos.head(10)


,Brand Name,Device Type,Model Name,Color,Selling Price,Original Price,Display,Rating (Out of 5),Strap Material,Average Battery Life (in days),Reviews
0,Xiaomi,FitnessBand,Smart Band 5,Black,2499,2999,AMOLED Display,4.1,Thermoplastic polyurethane,14,NaN
1,Xiaomi,FitnessBand,Smart Band 4,Black,2099,2499,AMOLED Display,4.2,Thermoplastic polyurethane,14,NaN
2,Xiaomi,FitnessBand,HMSH01GE,Black,1722,2099,LCD Display,3.5,Leather,14,NaN
3,Xiaomi,FitnessBand,Smart Band 5,Black,2469,2999,AMOLED Display,4.1,Thermoplastic polyurethane,14,NaN
4,Xiaomi,FitnessBand,Band 3,Black,1799,2199,OLED Display,4.3,Plastic,7,NaN
5,Xiaomi,FitnessBand,Band - HRX Edition,Black,1299,1799,OLED Display,4.2,Plastic,20,NaN
6,Xiaomi,FitnessBand,Band 2,Black,2499,2499,OLED Display,4.3,Plastic,7,NaN
7,Xiaomi,Smartwatch,Revolve,Black,12349,15999,AMOLED Display,4.4,Silicone,14,2.0
8,Xiaomi,Smartwatch,RevolveActive,Black,12999,15999,AMOLED Display,4.4,Silicone,14,3.0
9,Xiaomi,FitnessBand,Smart Band 3i,Black,1270,1599,OLED Display,4.2,Thermoplastic polyurethane,7,NaN


In [380]:
datos.dtypes

Brand Name                         object
Device Type                        object
Model Name                         object
Color                              object
Selling Price                       int32
Original Price                      int32
Display                            object
Rating (Out of 5)                 float64
Strap Material                     object
Average Battery Life (in days)      int64
Reviews                           float64
dtype: object

In [381]:
variablesNumericas = datos[["Selling Price","Original Price", "Rating (Out of 5)", "Average Battery Life (in days)"]] #TODO mirar si los missing en ratings y ABV influyen en el cálculo
mediaNumericas = round(variablesNumericas.mean(), 3)
stdNumericas = round(variablesNumericas.std(), 3)
minNumericas = round(variablesNumericas.min(), 3)
maxNumericas = round(variablesNumericas.max(), 3)

valoresInteres = pd.DataFrame([mediaNumericas,stdNumericas,minNumericas,maxNumericas], columns=["Selling Price","Original Price", "Rating (Out of 5)", "Average Battery Life (in days)"])
valoresInteres["Valor"] = ["Media", "Desviación Estándar", "Mínimo", "Máximo"]
valoresInteres.set_index("Valor", inplace = True)

valoresInteres

,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days)
Valor,,,,
Media,22110.373,25365.361,4.230,9.027
Desviación Estándar,19914.926,20384.029,0.391,7.869
Mínimo,1195.000,1599.000,2.000,1.000
Máximo,122090.000,122090.000,5.000,45.000


### 2. Hay datos que nos interesa analizar basándonos en agrupaciones, para darle un sentido a nuestro análisis en base a esa agrupación. Basándonos en las siguientes agrupaciones:
* Por tipo de dispositivo



In [382]:
tiposDisp = list(set(datos["Device Type"]))
print(tiposDisp)

['Smartwatch', 'FitnessBand']


* Por precio de venta. Estableceremos cuatro grupos en base a la media del precio de venta de cada tipo de dispositivo:
 * Smartwatches con un precio inferior o igual a la media de precios de venta de estos dispositivos
 * Smartwatches con un precio superior a la media de precios de venta de estos dispositivos
 * Fitnessbands con un precio inferior o igual a la media de precios de venta de estos dispositivos
 * Fitnessbands con un precio superior a la media de precios de venta de estos dispositivos

In [383]:
# Precio medio de un Smartwatch:

swMedia = datos.loc[datos["Device Type"]=="Smartwatch", "Selling Price"].mean()

# Precio medio de una FitnessBand:

fbMedia = datos.loc[datos["Device Type"]=="FitnessBand", "Selling Price"].mean()


# Condiciones a cumplir:

swBajoMedia = (datos["Selling Price"] <= swMedia) & (datos["Device Type"] == "Smartwatch")
swSobreMedia = (datos["Selling Price"] > swMedia) & (datos["Device Type"] == "Smartwatch")
fbBajoMedia = (datos["Selling Price"] <= fbMedia) & (datos["Device Type"] == "FitnessBand")
fbSobreMedia = (datos["Selling Price"] > fbMedia) & (datos["Device Type"] == "FitnessBand")

* Por marca

In [384]:
marcas = list(set(datos["Brand Name"]))
print(marcas)

['LCARE', 'Fastrack', 'APPLE', 'SAMSUNG ', 'Honor', 'Noise ', 'FOSSIL ', 'realme', 'Infinix', 'huami', 'LAVA', 'Noise', 'boAt ', 'Huawei', 'GARMIN ', 'Oppo', 'GOQii', 'FitBit', 'OnePlus ', 'Xiaomi']


Hay una marca, "Noise", duplicada con espacio, que tendremos que arreglar:

In [385]:
datos.loc[datos["Brand Name"]=="Noise ","Brand Name"] = "Noise"

marcas = list(set(datos["Brand Name"]))
print(marcas)

['LCARE', 'Fastrack', 'APPLE', 'SAMSUNG ', 'Honor', 'FOSSIL ', 'realme', 'Infinix', 'huami', 'LAVA', 'Noise', 'boAt ', 'Huawei', 'GARMIN ', 'Oppo', 'GOQii', 'FitBit', 'OnePlus ', 'Xiaomi']


Calcula los siguientes estadísticos en base a cada una de las agrupaciones definidas previamente con respecto a las variables selling price, original price, rating, average battery life in days y reviews:



In [386]:
datosFilt = datos[["Device Type", "Selling Price", "Original Price", "Rating (Out of 5)", "Average Battery Life (in days)", "Reviews"]]

* Número de observaciones

In [387]:
# Por tipo de dispositivo

print("Número de observaciones por dispositivo:")

datosPorTipo = datosFilt.groupby("Device Type").count()
datosPorTipo

Número de observaciones por dispositivo:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Device Type,,,,,
FitnessBand,75,75,75,75,13
Smartwatch,490,490,439,490,65


In [388]:
# Por precio de venta
print("Número de observaciones por precio de venta:")

datosswBajoMedia = datosFilt.loc[swBajoMedia].groupby("Device Type").count()
datosswBajoMedia.rename(index={'Smartwatch': 'Smartwatches bajo precio medio'}, inplace=True)

datosswSobreMedia = datosFilt.loc[swSobreMedia].groupby("Device Type").count()
datosswSobreMedia.rename(index={'Smartwatch': 'Smartwatches sobre precio medio'}, inplace=True)

datosfbBajoMedia = datosFilt.loc[fbBajoMedia].groupby("Device Type").count()
datosfbBajoMedia.rename(index={'FitnessBand': 'FitnessBands bajo precio medio'}, inplace=True)

datosfbSobreMedia = datosFilt.loc[fbSobreMedia].groupby("Device Type").count()
datosfbSobreMedia.rename(index={'FitnessBand': 'FitnessBands sobre precio medio'}, inplace=True)

datosPorPrecio = datosswBajoMedia.append(datosswSobreMedia).append(datosfbBajoMedia).append(datosfbSobreMedia)
datosPorPrecio.index.names = ['Grupos por precio de venta']

datosPorPrecio

Número de observaciones por precio de venta:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Grupos por precio de venta,,,,,
Smartwatches bajo precio medio,323,323,306,323,65
Smartwatches sobre precio medio,167,167,133,167,0
FitnessBands bajo precio medio,45,45,45,45,12
FitnessBands sobre precio medio,30,30,30,30,1


La categoría "FitnessBand prices above mean" no aparece dado que no existe ninguna FitnessBand en el dataset cuyo precio sea superior al precio medio:

In [389]:
datosfbSobreMedia = datosFilt.loc[fbSobreMedia].count()
datosfbSobreMedia

Device Type                       30
Selling Price                     30
Original Price                    30
Rating (Out of 5)                 30
Average Battery Life (in days)    30
Reviews                            1
dtype: int64

In [390]:
# Por marca
print("Número de observaciones por marca:")    
datosMarca = datos[["Brand Name", "Selling Price", "Original Price", "Rating (Out of 5)", "Average Battery Life (in days)", "Reviews"]].groupby("Brand Name").count()
datosMarca

Número de observaciones por marca:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Brand Name,,,,,
APPLE,86,86,86,86,0
FOSSIL,133,133,133,133,15
Fastrack,6,6,6,6,0
FitBit,51,51,51,51,0
GARMIN,101,101,50,101,0
GOQii,4,4,4,4,0
Honor,20,20,20,20,20
Huawei,26,26,26,26,0
Infinix,1,1,1,1,0


* Número de valores ausentes (missing)

In [391]:
#Por tipo de dispositivo

print("Número de ausencias por dispositivo:")
    
ausentesPorTipo = datosFilt.set_index("Device Type")
ausentesPorTipo = ausentesPorTipo.isna().groupby("Device Type").sum()
ausentesPorTipo

Número de ausencias por dispositivo:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Device Type,,,,,
FitnessBand,0,0,0,0,62
Smartwatch,0,0,51,0,425


In [392]:
# Por precio de venta
print("Número de ausencias por precio de venta:")

ausentesswBajoMedia = datosFilt.loc[swBajoMedia]
ausentesswBajoMedia = ausentesswBajoMedia.set_index("Device Type")
ausentesswBajoMedia = ausentesswBajoMedia.isna().groupby("Device Type").sum()
ausentesswBajoMedia.rename(index={'Smartwatch': 'Smartwatches bajo precio medio'}, inplace=True)

ausentesswSobreMedia = datosFilt.loc[swSobreMedia]
ausentesswSobreMedia = ausentesswSobreMedia.set_index("Device Type")
ausentesswSobreMedia = ausentesswSobreMedia.isna().groupby("Device Type").sum()
ausentesswSobreMedia.rename(index={'Smartwatch': 'Smartwatches sobre precio medio'}, inplace=True)

ausentesfbBajoMedia = datosFilt.loc[fbBajoMedia]
ausentesfbBajoMedia = ausentesfbBajoMedia.set_index("Device Type")
ausentesfbBajoMedia = ausentesfbBajoMedia.isna().groupby("Device Type").sum()
ausentesfbBajoMedia.rename(index={'FitnessBand': 'FitnessBands bajo precio medio'}, inplace=True)

ausentesfbSobreMedia = datosFilt.loc[fbSobreMedia]
ausentesfbSobreMedia = ausentesfbSobreMedia.set_index("Device Type")
ausentesfbSobreMedia = ausentesfbSobreMedia.isna().groupby("Device Type").sum()
ausentesfbSobreMedia.rename(index={'FitnessBand': 'FitnessBands sobre precio medio'}, inplace=True)

ausentesPorPrecio = ausentesswBajoMedia.append(ausentesswSobreMedia).append(ausentesfbBajoMedia).append(ausentesfbSobreMedia)
ausentesPorPrecio.index.names = ['Ausentes agrupados por precio de venta']

ausentesPorPrecio

Número de ausencias por precio de venta:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Ausentes agrupados por precio de venta,,,,,
Smartwatches bajo precio medio,0,0,17,0,258
Smartwatches sobre precio medio,0,0,34,0,167
FitnessBands bajo precio medio,0,0,0,0,33
FitnessBands sobre precio medio,0,0,0,0,29


Igual que sucedió con los datos de FitnessBands con precio superior a la media, tampoco hay valores missing en la misma categoría:

In [393]:
ausentesfbSobreMedia = datosFilt.loc[fbSobreMedia].count()
ausentesfbSobreMedia

Device Type                       30
Selling Price                     30
Original Price                    30
Rating (Out of 5)                 30
Average Battery Life (in days)    30
Reviews                            1
dtype: int64

In [394]:
# Por marca
print("Número de ausencias por marca:")

datosMarca = datos[["Brand Name", "Selling Price", "Original Price", "Rating (Out of 5)", "Average Battery Life (in days)", "Reviews"]]
ausentesMarca = datosMarca.set_index("Brand Name")
ausentesMarca = ausentesMarca.isna().groupby("Brand Name").sum()
ausentesMarca


Número de ausencias por marca:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Brand Name,,,,,
APPLE,0,0,0,0,86
FOSSIL,0,0,0,0,118
Fastrack,0,0,0,0,6
FitBit,0,0,0,0,51
GARMIN,0,0,51,0,101
GOQii,0,0,0,0,4
Honor,0,0,0,0,0
Huawei,0,0,0,0,26
Infinix,0,0,0,0,1


* Mediana

In [395]:
# Por tipo de dispositivo

medianaPorTipo = datosFilt.groupby("Device Type").median()
medianaPorTipo

,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Device Type,,,,,
FitnessBand,2999,3999,4.2,7,4628.0
Smartwatch,18990,21990,4.3,7,259.0


In [396]:
# Por precio de venta
print("Medianas por precio de venta:")

medianaswBajoMedia = datosFilt.loc[swBajoMedia].groupby("Device Type").median()
medianaswBajoMedia.rename(index={'Smartwatch': 'Smartwatches bajo precio medio'}, inplace=True)

medianaswSobreMedia = datosFilt.loc[swSobreMedia].groupby("Device Type").median()
medianaswSobreMedia.rename(index={'Smartwatch': 'Smartwatches sobre precio medio'}, inplace=True)

medianafbBajoMedia = datosFilt.loc[fbBajoMedia].groupby("Device Type").median()
medianafbBajoMedia.rename(index={'FitnessBand': 'FitnessBands bajo precio medio'}, inplace=True)

medianafbSobreMedia = datosFilt.loc[fbSobreMedia].groupby("Device Type").median()
medianafbSobreMedia.rename(index={'FitnessBand': 'FitnessBands sobre precio medio'}, inplace=True)

medianaPorPrecio = medianaswBajoMedia.append(medianaswSobreMedia).append(medianafbBajoMedia).append(medianafbSobreMedia)
medianaPorPrecio.index.names = ['Grupos por precio de venta']

medianaPorPrecio

Medianas por precio de venta:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Grupos por precio de venta,,,,,
Smartwatches bajo precio medio,13495.0,16999.0,4.2,7,259.0
Smartwatches sobre precio medio,40900.0,43900.0,4.5,7,NaN
FitnessBands bajo precio medio,2199.0,2999.0,4.1,7,4628.0
FitnessBands sobre precio medio,8994.5,14494.5,4.3,7,77.0


In [397]:
# Por marca
print("Medianas por marca:")

datosMarca = datos[["Brand Name", "Selling Price", "Original Price", "Rating (Out of 5)", "Average Battery Life (in days)", "Reviews"]].groupby("Brand Name").median()
datosMarca

Medianas por marca:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Brand Name,,,,,
APPLE,45690.0,49900.0,4.60,1.0,NaN
FOSSIL,13495.0,16495.0,4.30,2.0,46.0
Fastrack,1620.0,2095.0,4.05,4.0,NaN
FitBit,14990.0,15999.0,4.20,7.0,NaN
GARMIN,28990.0,31490.0,4.20,14.0,NaN
GOQii,3249.0,3249.0,3.40,7.0,NaN
Honor,2949.0,4249.0,4.30,12.0,4628.0
Huawei,11990.0,20490.0,4.25,14.0,NaN
Infinix,1499.0,2999.0,3.80,4.0,NaN


* Varianza (calcularemos la desviación estándar, que es más útil. Calcular la varianza sería lo mismo cambiando "std" por "var")

In [398]:
# Por tipo de dispositivo

stdPorTipo = round(datosFilt.groupby("Device Type").std(), 3) # TODO preguntar si se quiere varianza o std
stdPorTipo

,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Device Type,,,,,
FitnessBand,4428.854,6231.133,0.317,3.089,8418.510
Smartwatch,20138.247,20400.166,0.398,8.347,3771.094


In [399]:
# Por precio de venta
print("Desviación estándar por precio de venta:")

stdswBajoMedia = round(datosFilt.loc[swBajoMedia].groupby("Device Type").std(), 3)
stdswBajoMedia.rename(index={'Smartwatch': 'Smartwatches bajo precio medio'}, inplace=True)

stdswSobreMedia = round(datosFilt.loc[swSobreMedia].groupby("Device Type").std(), 3)
stdswSobreMedia.rename(index={'Smartwatch': 'Smartwatches sobre precio medio'}, inplace=True)

stdfbBajoMedia = round(datosFilt.loc[fbBajoMedia].groupby("Device Type").std(), 3)
stdfbBajoMedia.rename(index={'FitnessBand': 'FitnessBands bajo precio medio'}, inplace=True)

stdfbSobreMedia = round(datosFilt.loc[fbSobreMedia].groupby("Device Type").std(), 3)
stdfbSobreMedia.rename(index={'FitnessBand': 'FitnessBands sobre precio medio'}, inplace=True)

stdPorPrecio = stdswBajoMedia.append(stdswSobreMedia).append(stdfbBajoMedia).append(stdfbSobreMedia)
stdPorPrecio.index.names = ['Grupos por precio de venta']

stdPorPrecio

Desviación estándar por precio de venta:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Grupos por precio de venta,,,,,
Smartwatches bajo precio medio,6147.754,6681.278,0.396,6.035,3771.094
Smartwatches sobre precio medio,19448.227,20244.836,0.330,11.581,NaN
FitnessBands bajo precio medio,974.786,1593.549,0.360,3.666,8334.994
FitnessBands sobre precio medio,3508.320,5705.739,0.190,1.680,NaN


In [400]:
# Por marca
print("Desviaciones estándar por marca:")

datosMarca = round(datos[["Brand Name", "Selling Price", "Original Price", "Rating (Out of 5)", "Average Battery Life (in days)", "Reviews"]].groupby("Brand Name").std(), 3)
datosMarca

Desviaciones estándar por marca:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Brand Name,,,,,
APPLE,20606.442,20159.821,0.185,0.000,NaN
FOSSIL,5031.257,4241.272,0.392,5.240,146.426
Fastrack,495.282,496.655,0.117,2.510,NaN
FitBit,5787.428,6025.407,0.185,0.832,NaN
GARMIN,20748.706,22890.223,0.615,10.416,NaN
GOQii,853.913,853.913,0.480,0.000,NaN
Honor,4745.051,6859.975,0.376,5.609,7898.383
Huawei,13242.405,12827.351,0.325,3.473,NaN
Infinix,NaN,NaN,NaN,NaN,NaN


Comprobamos que las marcas Infinix y LAVA no tienen varianza asignada. Veamos si puede deberse a que no se puede calcular:

In [401]:
datos.loc[(datos["Brand Name"]=="Infinix") | (datos["Brand Name"]=="LAVA")]

,Brand Name,Device Type,Model Name,Color,Selling Price,Original Price,Display,Rating (Out of 5),Strap Material,Average Battery Life (in days),Reviews
88,Infinix,FitnessBand,Band 5,"Black, Blue, Red",1499,2999,TFT-LCD Display,3.8,Silicone,4,NaN
91,LAVA,FitnessBand,BeFit,Black,1999,3499,LCD Display,3.6,Thermoplastic polyurethane,7,NaN


Efectivamente, estas marcas solo tienen un dispositivo.

* Valores máximo y mínimo

In [402]:
# Por tipo de dispositivo
print("Mínimo por tipo de dispositivo:")
minPorTipo = datosFilt.groupby("Device Type").min() # TODO preguntar si se quiere varianza o std
minPorTipo

Mínimo por tipo de dispositivo:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Device Type,,,,,
FitnessBand,1195,1599,2.8,3,77.0
Smartwatch,1395,1995,2.0,1,2.0


In [403]:
print("Máximo por tipo de dispositivo:")
minPorTipo = datosFilt.groupby("Device Type").max() # TODO preguntar si se quiere varianza o std
minPorTipo

Máximo por tipo de dispositivo:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Device Type,,,,,
FitnessBand,16999,38500,4.7,20,23426.0
Smartwatch,122090,122090,5.0,45,23069.0


In [404]:
# Por precio de venta
print("Mínimos por precio de venta:")

minswBajoMedia = round(datosFilt.loc[swBajoMedia].groupby("Device Type").min(), 3)
minswBajoMedia.rename(index={'Smartwatch': 'Smartwatches bajo precio medio'}, inplace=True)

minswSobreMedia = round(datosFilt.loc[swSobreMedia].groupby("Device Type").min(), 3)
minswSobreMedia.rename(index={'Smartwatch': 'Smartwatches sobre precio medio'}, inplace=True)

minfbBajoMedia = round(datosFilt.loc[fbBajoMedia].groupby("Device Type").min(), 3)
minfbBajoMedia.rename(index={'FitnessBand': 'FitnessBands bajo precio medio'}, inplace=True)

minfbSobreMedia = round(datosFilt.loc[fbSobreMedia].groupby("Device Type").min(), 3)
minfbSobreMedia.rename(index={'FitnessBand': 'FitnessBands sobre precio medio'}, inplace=True)

minPorPrecio = minswBajoMedia.append(minswSobreMedia).append(minfbBajoMedia).append(minfbSobreMedia)
minPorPrecio.index.names = ['Grupos por precio de venta']

minPorPrecio

Mínimos por precio de venta:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Grupos por precio de venta,,,,,
Smartwatches bajo precio medio,1395,1995,2.0,1,2.0
Smartwatches sobre precio medio,24989,24990,3.2,1,NaN
FitnessBands bajo precio medio,1195,1599,2.8,3,122.0
FitnessBands sobre precio medio,5499,5499,3.8,5,77.0


In [405]:
print("Máximos por precio de venta:")

maxswBajoMedia = round(datosFilt.loc[swBajoMedia].groupby("Device Type").max(), 3)
maxswBajoMedia.rename(index={'Smartwatch': 'Smartwatches bajo precio medio'}, inplace=True)

maxswSobreMedia = round(datosFilt.loc[swSobreMedia].groupby("Device Type").max(), 3)
maxswSobreMedia.rename(index={'Smartwatch': 'Smartwatches sobre precio medio'}, inplace=True)

maxfbBajoMedia = round(datosFilt.loc[fbBajoMedia].groupby("Device Type").max(), 3)
maxfbBajoMedia.rename(index={'FitnessBand': 'FitnessBands bajo precio medio'}, inplace=True)

maxfbSobreMedia = round(datosFilt.loc[fbSobreMedia].groupby("Device Type").max(), 3)
maxfbSobreMedia.rename(index={'FitnessBand': 'FitnessBands sobre precio medio'}, inplace=True)

maxPorPrecio = maxswBajoMedia.append(maxswSobreMedia).append(maxfbBajoMedia).append(maxfbSobreMedia)
maxPorPrecio.index.names = ['Grupos por precio de venta']

maxPorPrecio

Máximos por precio de venta:


,Selling Price,Original Price,Rating (Out of 5),Average Battery Life (in days),Reviews
Grupos por precio de venta,,,,,
Smartwatches bajo precio medio,24495,34550,5.0,28,23069.0
Smartwatches sobre precio medio,122090,122090,5.0,45,NaN
FitnessBands bajo precio medio,5299,9990,4.4,20,23426.0
FitnessBands sobre precio medio,16999,38500,4.7,14,77.0


TODO: Conclusiones

## 3. Selecciona los dispositivos en los que la ratio de la duración de la batería con respecto al precio sea superior a la media de ratio de duración, que también debe calcularse. Comenta los resultados obtenidos.